<a href="https://colab.research.google.com/github/paoguilar/POO/blob/main/birthdayproyect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 36.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 2.9 MB/s eta 0:00:00


In [7]:
%%writefile app.py
# gestor_cumpleanos_app.py

import csv
import smtplib
import random
import os
from datetime import datetime
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import streamlit as st
import pandas as pd

# ------------------- MODELOS -------------------

class Contacto:
    """
    Clase que representa un contacto con nombre, fecha de nacimiento, correo
    y un mensaje personalizado opcional.
    """
    def __init__(self, nombre, fecha_nacimiento, correo, mensaje_personalizado=""):
        self.nombre = nombre
        self.fecha_nacimiento = datetime.strptime(fecha_nacimiento, "%Y-%m-%d")
        self.correo = correo
        self.mensaje_personalizado = mensaje_personalizado

    def dias_para_cumple(self):
        """Calcula cuántos días faltan para el próximo cumpleaños."""
        hoy = datetime.now()
        proximo = self.fecha_nacimiento.replace(year=hoy.year)
        if proximo < hoy:
            proximo = proximo.replace(year=hoy.year + 1)
        return (proximo - hoy).days

# ------------------- GESTOR -------------------

class GestorCumpleanos:
    """
    Clase principal que gestiona los contactos, mensajes y el envío de correos.
    Se encarga de cargar y guardar datos en archivos CSV.
    """
    def __init__(self, archivo_contactos="contactos.csv", archivo_mensajes="mensajes.csv"):
        self.archivo_contactos = archivo_contactos
        self.archivo_mensajes = archivo_mensajes
        self.contactos = self.cargar_contactos()
        self.mensajes = self.cargar_mensajes()

    def cargar_contactos(self):
        contactos = []
        if os.path.exists(self.archivo_contactos):
            with open(self.archivo_contactos, newline='', encoding='utf-8') as f:
                reader = csv.DictReader(f)
                for row in reader:
                    contactos.append(Contacto(row["nombre"], row["fecha_nacimiento"], row["correo"], row.get("mensaje_personalizado", "")))
        return contactos

    def guardar_contacto(self, contacto):
        archivo_nuevo = not os.path.exists(self.archivo_contactos)
        with open(self.archivo_contactos, mode='a', newline='', encoding='utf-8') as f:
            fieldnames = ["nombre", "fecha_nacimiento", "correo", "mensaje_personalizado"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            if archivo_nuevo:
                writer.writeheader()
            writer.writerow({
                "nombre": contacto.nombre,
                "fecha_nacimiento": contacto.fecha_nacimiento.strftime("%Y-%m-%d"),
                "correo": contacto.correo,
                "mensaje_personalizado": contacto.mensaje_personalizado
            })
        self.contactos.append(contacto)

    def cargar_mensajes(self):
        mensajes = []
        if os.path.exists(self.archivo_mensajes):
            with open(self.archivo_mensajes, newline='', encoding='utf-8') as f:
                reader = csv.reader(f)
                for row in reader:
                    if row:
                        mensajes.append(row[0])
        return mensajes

    def guardar_mensaje(self, mensaje):
        with open(self.archivo_mensajes, mode='a', newline='', encoding='utf-8') as f:
            writer = csv.writer(f)
            writer.writerow([mensaje])
        self.mensajes.append(mensaje)

    def obtener_mensaje(self, contacto):
        if contacto.mensaje_personalizado:
            return contacto.mensaje_personalizado
        if self.mensajes:
            return random.choice(self.mensajes)
        return "🎉 ¡Feliz cumpleaños! 🎂"

    def enviar_correo(self, remitente, clave_app, contacto):
        mensaje = self.obtener_mensaje(contacto)
        msg = MIMEMultipart()
        msg["From"] = remitente
        msg["To"] = contacto.correo
        msg["Subject"] = f"🎉 ¡Feliz cumpleaños, {contacto.nombre}! 🎈"

        msg.attach(MIMEText(mensaje, "plain"))

        try:
            with smtplib.SMTP("smtp.gmail.com", 587) as servidor:
                servidor.starttls()
                servidor.login(remitente, clave_app)
                servidor.send_message(msg)
        except Exception as e:
            print(f"Error al enviar a {contacto.nombre}: {e}")

    def enviar_correos_hoy(self, remitente, clave_app):
        hoy = datetime.now().strftime("%m-%d")
        for contacto in self.contactos:
            if contacto.fecha_nacimiento.strftime("%m-%d") == hoy:
                self.enviar_correo(remitente, clave_app, contacto)

# ------------------- INTERFAZ -------------------

gestor = GestorCumpleanos()

st.title("🎂 Gestor de Cumpleaños 🎁")

menu = ["➕ Registrar", "📅 Cuenta regresiva", "✉️ Mensajes", "📬 Enviar correos"]
opcion = st.sidebar.selectbox("📌 Menú", menu)

if opcion == "➕ Registrar":
    st.header("Registrar nuevo cumpleaños 🎈")
    nombre = st.text_input("👤 Nombre")
    fecha = st.date_input("📆 Fecha de nacimiento", min_value=datetime(1900, 1, 1))
    correo = st.text_input("📧 Correo electrónico")
    mensaje = st.text_area("📝 Mensaje personalizado (opcional)")
    if st.button("Guardar 🎀"):
        contacto = Contacto(nombre, fecha.strftime("%Y-%m-%d"), correo, mensaje)
        gestor.guardar_contacto(contacto)
        st.success("✅ Contacto guardado")

elif opcion == "📅 Cuenta regresiva":
    st.header("🎯 Próximos cumpleaños")
    datos = [{
        "Nombre": c.nombre,
        "Correo": c.correo,
        "Fecha": c.fecha_nacimiento.strftime("%Y-%m-%d"),
        "Días restantes": c.dias_para_cumple()
    } for c in gestor.contactos]
    st.dataframe(pd.DataFrame(datos).sort_values("Días restantes"))

elif opcion == "✉️ Mensajes":
    st.header("✍️ Mensajes de felicitación")
    nuevo = st.text_area("➕ Nuevo mensaje")
    if st.button("Guardar mensaje 💾"):
        gestor.guardar_mensaje(nuevo)
        st.success("✅ Mensaje guardado")
    if gestor.mensajes:
        st.subheader("📋 Mensajes existentes")
        for i, m in enumerate(gestor.mensajes, 1):
            st.markdown(f"**{i}.** {m}")

elif opcion == "📬 Enviar correos":
    st.header("🚀 Enviar felicitaciones")
    remitente = st.text_input("📨 Correo remitente (Gmail)")
    clave = st.text_input("🔑 Clave de aplicación", type="password")
    if st.button("Enviar correos hoy 🎉"):
        gestor.enviar_correos_hoy(remitente, clave)
        st.success("✅ Correos enviados si había cumpleaños hoy")


Overwriting app.py


In [3]:
!npm install -g localtunnel

⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙
added 22 packages in 2s
⠙
⠙3 packages are looking for funding
⠙  run `npm fund` for details
⠙

In [ ]:
!streamlit run app.py &>/content/logs.txt & npx localtunnel --port 8501 & curl ipv4.icanhazip.com

34.69.61.88
⠙⠹⠸⠼⠴⠦your url is: https://shaggy-taxes-mix.loca.lt
